In [16]:
# GPU NVIDIA 
!nvidia-smi

Sat Jul 24 15:38:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Concetar ao Driver**

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'1_e-MEC Protocolado UniaoSaoRoque.zip'    InformeRendimentos.pdf
 APKs					   Inglês
'Apresentacao_Felipe_03_Jul(1).pdf'	  'IP Webcam'
'Artigos Juliano.gsheet'		   Livros
'Artigos Mediana de tukey'		  'me (1).jpg'
 BitBucket.rar				   Meganha
 Classroom				   me.jpg
'Colab Notebooks'			   Mestrado
 cond_julho.pdf				  'MF App'
'[CTRL+Play]'				  'My Drive'
'curriculo atualizado.docx'		   Orientação_Juliano.drawio
'curriculo Ingles (1).docx'		  'Palestras & Minicursos'
 Desktop.rar				  'Projeto OS.drawio'
'Documento de Felipe Meganha.pdf'	   semanaEngenharia.rar
'Doutorado - Pesquisa Cientifica.drawio'  'TCC Cinthya'
 Doutorado.txt				  '[Udemy] - YOLO_v3'
 HaarCascades.gdoc			  'Untitled Diagram.drawio'
 IC					   WhatsApp.drawio
'[IFET]Informatica Básica'		   yolov3
 IFTTT


**1) Clone do repositório Darknet**



In [18]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compilando Darknet usando Nvidia GPU**


In [19]:
# Habilitando a GPU e o OPENCV no makefile
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

**3) Configurar Darknet para o treinamento do YOLO V3**

In [20]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [21]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [22]:
# Crie uma pasta no google drive para que possamos salvar lá os pesos
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [23]:
!echo "Sheep" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [24]:
# Baixar pesos modelo darknet 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-24 15:40:53--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  64.8MB/s    in 2.4s    

2021-07-24 15:40:56 (64.8 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



**4) Extração das imagens**

As imagens precisam estar dentro de um arquivo zip chamado "images.zip" e precisam estar dentro da pasta "yolov3" no Google Drive

In [25]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace data/obj/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/classes.txt    
 extracting: data/obj/sheep0.png     
 extracting: data/obj/sheep0.txt     
 extracting: data/obj/sheep1.png     
  inflating: data/obj/sheep1.txt     
 extracting: data/obj/sheep10.png    
  inflating: data/obj/sheep10.txt    
 extracting: data/obj/sheep100.png   
  inflating: data/obj/sheep100.txt   
 extracting: data/obj/sheep101.png   
  inflating: data/obj/sheep101.txt   
 extracting: data/obj/sheep102.png   
 extracting: data/obj/sheep102.txt   
 extracting: data/obj/sheep103.png   
  inflating: data/obj/sheep103.txt   
 extracting: data/obj/sheep104.png   
 extracting: data/obj/sheep104.txt   
 extracting: data/obj/sheep105.png   
  inflating: data/obj/sheep105.txt   
 extracting: data/obj/sheep106.png   
  inflating: data/obj/sheep106.txt

In [26]:
# Vamos converter o índice da classe nos arquivos .txt. Como estamos trabalhando com apenas uma classe, supõe-se que seja a classe 0.
# Se o índice for diferente de 0, vamos alterá-lo.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # Tamanho da Imagem
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Definindo as coordenadas
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Salvando
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.212121 0.562092 0.248485 0.392157

['15', '0.212121', '0.562092', '0.248485', '0.392157']
0 data/obj/sheep174.txt
0 0.212121 0.562092 0.248485 0.392157
15 0.427273 0.549020 0.218182 0.418301

['15', '0.427273', '0.549020', '0.218182', '0.418301']
0 data/obj/sheep174.txt
0 0.427273 0.549020 0.218182 0.418301
15 0.645455 0.522876 0.212121 0.418301

['15', '0.645455', '0.522876', '0.212121', '0.418301']
0 data/obj/sheep174.txt
0 0.645455 0.522876 0.212121 0.418301
15 0.813636 0.535948 0.245455 0.418301

['15', '0.813636', '0.535948', '0.245455', '0.418301']
0 data/obj/sheep174.txt
0 0.813636 0.535948 0.245455 0.418301
15 0.378750 0.544610 0.277500 0.791822

['15', '0.378750', '0.544610', '0.277500', '0.791822']
1 data/obj/sheep148.txt
0 0.378750 0.544610 0.277500 0.791822
15 0.605000 0.559480 0.270000 0.799257

['15', '0.605000', '0.559480', '0.270000', '0.799257']
1 data/obj/sheep148.txt
0 0.605000 0.559480 0.270000 0.799257
15 0.421250 0.592105 0.207500 0.718045

['15', '0.421250',

In [27]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/sheep131.png', 'data/obj/sheep6.png', 'data/obj/sheep10.png', 'data/obj/sheep189.png', 'data/obj/sheep84.png', 'data/obj/sheep39.png', 'data/obj/sheep162.png', 'data/obj/sheep170.png', 'data/obj/sheep92.png', 'data/obj/sheep176.png', 'data/obj/sheep111.png', 'data/obj/sheep166.png', 'data/obj/sheep151.png', 'data/obj/sheep154.png', 'data/obj/sheep121.png', 'data/obj/sheep54.png', 'data/obj/sheep201.png', 'data/obj/sheep38.png', 'data/obj/sheep134.png', 'data/obj/sheep71.png', 'data/obj/sheep80.png', 'data/obj/sheep29.png', 'data/obj/sheep157.png', 'data/obj/sheep32.png', 'data/obj/sheep197.png', 'data/obj/sheep26.png', 'data/obj/sheep63.png', 'data/obj/sheep174.png', 'data/obj/sheep107.png', 'data/obj/sheep43.png', 'data/obj/sheep34.png', 'data/obj/sheep30.png', 'data/obj/sheep142.png', 'data/obj/sheep50.png', 'data/obj/sheep195.png', 'data/obj/sheep139.png', 'data/obj/sheep81.png', 'data/obj/sheep123.png', 'data/obj/sheep36.png', 'data/obj/sheep18.png', 'data/obj/sheep182.p

In [28]:
#Criando arquivo training.txt
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Treinamento**

In [ ]:

!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show